In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

import time
from sklearn.model_selection import KFold

In [2]:
# Model params
NAME = f"MLP-at{int(time.time())}"
lr = 0.001
optimizer = Adam(lr=lr)
EPOCHS = 600
BATCH_SIZE = 1054

In [3]:
# Load Data
features = np.loadtxt("./features2.csv", delimiter=';')

In [4]:
X = np.array([x[0:16] for x in features])
y = np.array([int(x[16])for x in features])
print(X[0], y[0])

[0.099358 0.007125 0.       0.281879 0.001205 0.095767 0.167785 0.201342
 0.249405 0.033774 0.       0.201342 0.056237 0.058088 0.100671 0.14094 ] 0


In [5]:
# Dataset Variables
num_features = 16
num_classes = 3

In [6]:
def create_model(X_train):
    model = Sequential()

    model.add(Dense(units=15, kernel_initializer="uniform", bias_initializer="zeros", input_shape=X_train.shape[1:], activation="relu"))

    model.add(Dense(units=num_classes, kernel_initializer="uniform", bias_initializer="zeros", activation="softmax"))

    model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"])

    return model

In [11]:
tensorboard = TensorBoard(
    log_dir=f"logs/{NAME}",
    histogram_freq=1,
    write_graph=True
    )

filepath = "models/MLP-{epoch:02d}.model"
checkpoint_dir = os.path.dirname(filepath)

#checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_acc", verbose = 0, save_best_only = True, mode = "max"))
#checkpoint = ModelCheckpoint(checkpoint_dir , monitor="val_acc", verbose = 0, save_best_only = True, mode = "max")


# construct the callback to save only the *best* model to disk
# based on the validation loss
path =  os.path.sep.join(["models", time.strftime("at%M%H_%d%m%Y")])
if not os.path.exists(path):
    os.makedirs(path)
fname = os.path.sep.join([path,	"weights-{epoch:03d}-{val_loss:.4f}.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
	save_best_only=True, verbose=1)


kfold = KFold(n_splits=5, shuffle=False)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(X, y):
    # Create model
    model = create_model(X[train])

    print("------------------------------------")
    print(f"Fitting Fold no. {fold_no}")
    model.fit(
        X[train], y[train],
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X[test], y[test]),
        #callbacks=[checkpoint]#[tensorboard, checkpoint]
    )

    results = model.evaluate(X[test], y[test])
    print(f"Score for {fold_no}: {results}")

    # Push fold loss
    loss_per_fold.append(results[0])

    # Push fold acc
    acc_per_fold.append(results[1])

    fold_no = fold_no + 1

print('--------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('--------------------------------------------------------------')
    print(f"> Fold {i+1} - Accuracy: {acc_per_fold[i]}%")
print('--------------------------------------------------------------')
print("Average scores for all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})")
print('--------------------------------------------------------------')

------------------------------------
Fitting Fold no. 1
Epoch 1/600
5/5 [==============================] - 0s 21ms/step - loss: 1.0984 - accuracy: 0.3527 - val_loss: 1.1011 - val_accuracy: 0.2558
Epoch 2/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0967 - accuracy: 0.3527 - val_loss: 1.1021 - val_accuracy: 0.2558
Epoch 3/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0940 - accuracy: 0.3527 - val_loss: 1.0983 - val_accuracy: 0.2558
Epoch 4/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0892 - accuracy: 0.3527 - val_loss: 1.0907 - val_accuracy: 0.2558
Epoch 5/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0825 - accuracy: 0.3527 - val_loss: 1.0834 - val_accuracy: 0.2762
Epoch 6/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0739 - accuracy: 0.4032 - val_loss: 1.0707 - val_accuracy: 0.6126
Epoch 7/600
5/5 [==============================] - 0s 3ms/step - loss: 1.0638 - accuracy: 0.5694 - val_loss: 

KeyboardInterrupt: 